In [13]:
import os
import pathlib
import sys
import matplotlib.pyplot as plt
from scipy import linalg


sys.path.append(os.path.join(pathlib.Path(os.getcwd()).parent))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from utils.bq_conn import bq_conn_file
# %%
bq_client = bq_conn_file(project_id="dsa-miad-365616")

In [15]:
pd.options.display.max_columns = None

query = '''
SELECT
    *
FROM `Proyecto.DatosSecopI`
ORDER BY anno_cargue_secop DESC
'''
bq_client = bq_conn_file(project_id="dsa-miad-365616")
secop_I = bq_client.execute_query(query=query)
print(f"El número de filas del dataset: {secop_I.shape[0]}\nEl número de columnas del dataset: {secop_I.shape[1]}")
secop_I.head(5)

El número de filas del dataset: 26553
El número de columnas del dataset: 38


,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,dias_ejec_final_contrato,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
0,191389136418143424,2019,13,contratacion minima cuantia,celebrado,50000000,contratar servicio suministro elementos materi...,suministro,facatativa,facatativa,cundinamarca,CPMMSFFAMCNo01DE2019,CPMMSFFAMCNO01DE2019,50000000,E,productos de uso final,5022,productos de cereales legumbres,502213,harina productos de molinos,8143424,nit de persona juridica,900791326,inversiones ocazo,cedula de ciudadania,1022363447,330,330,D,0,0,50000000,0,0,50000000,peso colombiano,contratar compra elementos materia prima para ...,{'url': 'https://www.contratos.gov.co/consulta...
1,1912101041779226352,2019,12,contratacion directa ley de,celebrado,53000000,adquisicion uniformes para dotar personal cuer...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA184DE2019,184DE2019,22980096902,E,productos de uso final,5310,ropa,531027,uniformes,9226352,nit de persona juridica,860020227,fondo rotatorio de la policia forpo,nit de persona juridica,860020227,660,690,D,1,30,22980096902,0,0,22980096902,peso colombiano,adquisicion uniformes para dotar personal cuer...,{'url': 'https://www.contratos.gov.co/consulta...
2,1912101443259265538,2019,12,contratacion directa ley de,celebrado,55000000,contratar diseno produccion tarjetas presentac...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA188DE2019,188DE2019,35998452,E,productos de uso final,5510,medios impresos,551015,publicaciones impresas,9265538,nit de persona juridica,830001113,imprenta nacional de colombia,nit de persona juridica,830001113,31,71,D,1,40,35998452,0,0,35998452,peso colombiano,contratar diseno produccion tarjetas presentac...,{'url': 'https://www.contratos.gov.co/consulta...
3,191294985788644918,2019,12,contratacion directa ley de,liquidado,72000000,prestar servicio mantenimiento preventivo corr...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA106DE2019,106DE2019,16767536,F,servicios,7210,servicios de mantenimiento reparaciones de con...,721015,servicios de apoyo para la construccion,8644918,nit de persona juridica,860025639,mitsubishi electric de colombia limitada,nit de persona juridica,860025639,170,170,D,0,0,16767536,0,0,16767536,peso colombiano,prestar servicio mantenimiento preventivo corr...,{'url': 'https://www.contratos.gov.co/consulta...
4,191988590538059861,2019,19,asociacion publico privada,celebrado,73000000,desintegracion vehiculos automotores estado in...,no encontrado en hoja proceso,bogota,bogota,bogota,CONVENIODEASOCIACIÓN231DE2018,231DE2018,0,F,servicios,7318,servicios de maquinado procesado,731810,servicios de maquinado,8059861,nit de persona juridica,806011019,recuperaciones naranjo recycling sas,cedula de ciudadania,57439563,360,360,D,0,0,0,0,0,0,peso colombiano,desintegracion vehiculos automotores estado in...,{'url': 'https://www.contratos.gov.co/consulta...


In [16]:
secopI_filter = secop_I[['id_modalidad','estado_del_proceso','id_objeto_a_contratar','tipo_de_contrato','departamento_ejecucion','municipios_ejecucion_','cuantia_proceso','id_grupo',
'id_familia','id_clase','plazo_de_ejec_del_contrato_','tiempo_adiciones_en_dias','dias_ejec_final_contrato']]

secopI_filter

,id_modalidad,estado_del_proceso,id_objeto_a_contratar,tipo_de_contrato,departamento_ejecucion,municipios_ejecucion_,cuantia_proceso,id_grupo,id_familia,id_clase,plazo_de_ejec_del_contrato_,tiempo_adiciones_en_dias,dias_ejec_final_contrato
0,13,celebrado,50000000,suministro,cundinamarca,facatativa,50000000,E,5022,502213,330,0,330
1,12,celebrado,53000000,prestacion de servicios,bogota,bogota,22980096902,E,5310,531027,660,30,690
2,12,celebrado,55000000,prestacion de servicios,bogota,bogota,35998452,E,5510,551015,31,40,71
3,12,liquidado,72000000,prestacion de servicios,bogota,bogota,16767536,F,7210,721015,170,0,170
4,19,celebrado,73000000,no encontrado en hoja proceso,bogota,bogota,0,F,7318,731810,360,0,360
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26548,12,celebrado,94000000,prestacion de servicios,cordoba,tierralta,1220210,F,9412,941216,2,0,2
26549,13,liquidado,94000000,prestacion de servicios,antioquia,puerto triunfo,2581895,F,9412,941218,30,0,30
26550,13,celebrado,94000000,prestacion de servicios,boyaca,santa rosa de viterbo,866525,F,9412,941215,30,0,30
26551,13,celebrado,95000000,obra,cauca,popayan,25538052,G,9512,951217,30,0,30


In [17]:
secopI_filter.columns

Index(['id_modalidad', 'estado_del_proceso', 'id_objeto_a_contratar',
       'tipo_de_contrato', 'departamento_ejecucion', 'municipios_ejecucion_',
       'cuantia_proceso', 'id_grupo', 'id_familia', 'id_clase',
       'plazo_de_ejec_del_contrato_', 'tiempo_adiciones_en_dias',
       'dias_ejec_final_contrato'],
      dtype='object')

In [18]:
base = secopI_filter

In [19]:
n = base.isna()== True 
base = base.fillna(0)
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26553 entries, 0 to 26552
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id_modalidad                 26553 non-null  Int64 
 1   estado_del_proceso           26553 non-null  object
 2   id_objeto_a_contratar        26553 non-null  Int64 
 3   tipo_de_contrato             26553 non-null  object
 4   departamento_ejecucion       26553 non-null  object
 5   municipios_ejecucion_        26553 non-null  object
 6   cuantia_proceso              26553 non-null  Int64 
 7   id_grupo                     26553 non-null  object
 8   id_familia                   26553 non-null  object
 9   id_clase                     26553 non-null  object
 10  plazo_de_ejec_del_contrato_  26553 non-null  Int64 
 11  tiempo_adiciones_en_dias     26553 non-null  Int64 
 12  dias_ejec_final_contrato     26553 non-null  Int64 
dtypes: Int64(6), object(7)
memory u

In [20]:
# Codificación de las variables categoricas
base['estado_del_proceso'] = pd.factorize(base.estado_del_proceso)[0]
base['tipo_de_contrato'] = pd.factorize(base.tipo_de_contrato)[0]
base['departamento_ejecucion'] = pd.factorize(base.departamento_ejecucion)[0]
base['id_grupo'] = pd.factorize(base.id_grupo)[0]
base['id_familia'] = pd.factorize(base.id_familia)[0]
base['id_clase'] = pd.factorize(base.id_clase)[0]
base['plazo_de_ejec_del_contrato_'] = pd.factorize(base.plazo_de_ejec_del_contrato_)[0]

In [ ]:
df = base
y_total = df.loc[:,df.columns=="tiempo_adiciones_en_dias"]
X_total = df.loc[:,df.columns!="tiempo_adiciones_en_dias"]
y_total = y_total.reset_index(drop= True)

# Reparto de datos en train y test
y_totalf = y_total.tiempo_adiciones_en_dias.tolist()
X_train, X_test, y_train, y_test = train_test_split(X_total, y_totalf, test_size = 0.33, random_state = 1)

In [ ]:
# Importar bibliotecas necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed(25)
tf.random.set_seed(25)

# Crear modelo
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(X_train.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compilar modelo
model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(0.001), metrics=['mae', 'mse'])

# Entrenar modelo
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=0)

# Evaluar modelo con datos de prueba
loss, mae, mse = model.evaluate(X_test, y_test, verbose=2)

# Hacer predicciones con el modelo
predictions = model.predict(X_test)

# Imprimir resultados
print("Error absoluto medio (MAE): {:.2f}".format(mae))
print("Error cuadrático medio (MSE): {:.2f}".format(mse))